In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#function that will filter out error triangles
def generate_neighborhood(triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def set_search_first(starting_node, neighborhood):
    """
    Modified Depth-First-Search utilizing sets to reduce duplicate checks:

    Neighborhood must be a dict with the keys being the vertex indices!
    """    
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return list(visited_nodes)
def get_connected_portions(neighborhood):
    neighborhood_copy = neighborhood.copy()
    portions = []
    while len(neighborhood_copy) > 0:
        starting_node = next(iter(neighborhood_copy))
        portion = set_search_first(starting_node, neighborhood_copy)
        for node in portion:
            neighborhood_copy.pop(node)
        portions.append(portion)
    return portions

def get_largest_portion_index(portions):
    portion_lengths = [len(portion) for portion in portions]
    return portion_lengths.index(max(portion_lengths))

def get_largest_portion(portions):
    return portions[get_largest_portion_index(portions)]

def remove_floating_artifacts(mesh,key):    
    mesh_copy = mesh.copy()
    
    #get the labels for the mesh
    #find out if in Orphan Table or Regular Neuron Table
    if len(ta3p100.CoarseLabelFinal() & key) > 0:
        mesh_labels = (ta3p100.CoarseLabelFinal & key).fetch1()
    elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
        mesh_labels = (ta3p100.CoarseLabelOrphan & key).fetch1()
    else:
        raise Exception('neuron' + str(key["segment_id"]) + 
                        'not present in any labels!')

    
    #look for errors
    not_errors = [i for i,k in enumerate(mesh_labels["triangles"]) if k != 10]
    original_triangles = mesh["triangles"]
    """
    print(type(not_errors))
    print(len(not_errors))
    print("not_errors = "+ str(not_errors[:100]))
    print(type(original_triangles))
    print(len(original_triangles))
    #print(original_triangles)
    print("not_errors = " + str(original_triangles[not_errors]))
    """
    
    mesh_copy['triangles'] = np.array(original_triangles[not_errors])
    
    return mesh_copy


def remove_isolated_vertices(mesh):
    mesh_copy = mesh.copy()

    neighborhood = generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    isolated_nodes = [portion.pop() for portion in get_connected_portions(neighborhood) if len(portion) == 1]

    vertices = mesh_copy['vertices']
    triangles = mesh_copy['triangles']
    vertex_list = list(vertices)

    if len(isolated_nodes) > 0:
        num_isolated_nodes_passed = 0
        isolated_nodes_set = set(isolated_nodes)
        count_to_decrement = np.zeros(len(vertices))
        for i in range(len(vertices)):
            if i in isolated_nodes_set:
                num_isolated_nodes_passed += 1
            else:
                count_to_decrement[i] = num_isolated_nodes_passed

        for i, triangle in enumerate(triangles):
            start = time.time()
            node1, node2, node3 = triangle
            triangles[i][0] -= count_to_decrement[node1]
            triangles[i][1] -= count_to_decrement[node2]
            triangles[i][2] -= count_to_decrement[node3]
        for i, isolated_node in enumerate(isolated_nodes):
            vertex_list.pop(isolated_node - i)

    mesh_copy['vertices'] = np.array(vertex_list)

    return mesh_copy


def remove_error_segments(key):
    
    
    full_start = time.time()

    print(str(key['segment_id']) +  ":")
    start = time.time()

    #find out if in Orphan Table or Regular Neuron Table
    if len(ta3p100.CoarseLabelFinal() & key) > 0:
        mesh = (ta3p100.CleansedMesh & key).fetch1()
    elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
        mesh = (ta3p100.CleansedMeshOrphan & key).fetch1()
    else:
        raise Exception('neuron' + str(key["segment_id"]) + 
                        'not present in any labels!')
    
    print(key['segment_id'], "mesh fetched.", time.time() - start)
    start = time.time()
    
    neighborhood = generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
    print(key['segment_id'] , "neighborhood generated.", time.time() - start)
    start = time.time()
    
    mesh = remove_floating_artifacts(mesh,key)
    print(key['segment_id'], "floating artifacts removed.", time.time() - start)
    start = time.time()

    mesh = remove_isolated_vertices(mesh)
    print(key['segment_id'], "isolated nodes removed.", time.time() - start)
    start = time.time()

    key['n_vertices'] = len(mesh['vertices'])
    key['n_triangles'] = len(mesh['triangles'])
    key['vertices'] = mesh['vertices']
    key['triangles'] = mesh['triangles']

    #self.insert1(key, skip_duplicates=True)
    print(key['segment_id'], "key successfully filtered.", time.time() - start)
    start = time.time()

    print("This took ", time.time() - full_start, "seconds.")
    print()
    return key

In [4]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)




In [5]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)




In [6]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp_pymesh"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [7]:
def meshlab_fix_manifold(key):
    
    file_loc = pathlib.Path.cwd() / "temp_pymesh"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [8]:
def meshlab_generic(key,meshlab_script="0",input_mesh="0",output_mesh="0"):
    
    file_loc = pathlib.Path.cwd() / "temp_pymesh"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    if input_mesh == "0":
        input_mesh = path_and_filename + ".off"
    
    if output_mesh=="0":
        output_mesh = path_and_filename+"_mls.off"
    
    if meshlab_script == "0":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlab script")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
        
    return output_mesh
    

In [9]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [ ]:

def make_skeleton(key):
    global_time = time.time()
    #get the mesh with the error segments filtered away
    start_time = time.time()
    new_key = remove_error_segments(key)
    print(f"Step 1: Removing error segments: {time.time() - start_time}")


    #print("Step 2: Remove all error semgents")
    start_time = time.time()
    #pass the vertices and faces to pymeshfix to become watertight
    meshfix = pymeshfix.MeshFix(new_key["vertices"],new_key["triangles"])
    meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
    print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

    #print("Step 2: Writing Off File")
    start_time = time.time()
    #write the new mesh to off file
    path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]),meshfix.v,meshfix.f)
    print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")

    #Need to filter it using meshlab server
    
    #take care of the non-manifold
    mid_ouptut = path_and_filename + "_mid_non_manifold.off"
    meshlab_script_mid = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    mid_output_return = meshlab_generic(key,meshlab_script_mid,mid_ouptut,)
    path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"] + "_non_manifold"),meshfix.v,meshfix.f)
    
    
    #Cleanse the mesh
    mid_ouptut = path_and_filename + "_mls_no_float.off"
    meshlab_script_output = str(pathlib.Path.cwd()) + "/" + "remove_floating_pieces.mls"
    
    mesh_output = meshlab_generic(key,meshlab_script_output,mid_ouptut)
   
    #Run the meshlabserver scripts
    start_time = time.time()
    output_mesh = meshlab_fix_manifold(key)
    print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")

    
    print(output_mesh[:-4])
    #send to be skeletonized
    start_time = time.time()
    return_value = cm.calcification(output_mesh[:-4])
    if return_value > 0:
        print('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                        ' did not finish... exited with error code: ' + str(return_value))
    print(f"Step 4: Generating Skeleton: {time.time() - start_time}")



#     #read in the skeleton files into an array
#     #start_time = time.time()
#     bone_array = read_skeleton_flat(path_and_filename+"_skeleton.cgal")
#     #print(bone_array)
#     if len(bone_array) <= 0:
#         raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
#     print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")


#     start_time = time.time()
#     new_key["n_branches"] = len(bone_array)
#     new_key["branches"] = bone_array
#     #new_key["branches"] = []


#     key["time_updated"]=str(datetime.datetime.now())
#     #print(key)
#     #if all goes well then write to database
#     self.insert1(key)
#     os.system("rm "+str(path_and_filename)+"*")
#     print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
#     print(f"Total time: {time.time() - global_time}")
#     print("\n\n")
          
                         
                                    

In [ ]:
ta3p100.Neurite()

In [10]:
ID = 648518346349479929
ID = 648518346349483124

In [11]:
key = dict(segment_id=ID,segmentation=2)

In [12]:
global_time = time.time()
#get the mesh with the error segments filtered away
start_time = time.time()
new_key = remove_error_segments(key)
print(f"Step 1: Removing error segments: {time.time() - start_time}")


#print("Step 2: Remove all error semgents")
start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight
meshfix = pymeshfix.MeshFix(new_key["vertices"],new_key["triangles"])
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=True)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]) + "_meshfix",meshfix.v,meshfix.f)
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")


                                    

648518346349483124:
648518346349483124 mesh fetched. 0.2649877071380615
648518346349483124 neighborhood generated. 9.26309871673584
648518346349483124 floating artifacts removed. 2.556729793548584
648518346349483124 isolated nodes removed. 12.330775737762451
648518346349483124 key successfully filtered. 6.198883056640625e-06
This took  24.418922662734985 seconds.

Step 1: Removing error segments: 24.693089723587036
Step 2: Pymesh shrinkwrapping: 115.86689400672913
Done writing OFF file
Step 3: Writing shrinkwrap off file: 4.045577049255371


In [ ]:
meshlab_fix_manifold(key)

In [ ]:
#Need to filter it using meshlab server
    
start_time = time.time()
#take care of the non-manifold
mid_ouptut = path_and_filename + "_mid_non_manifold.off"
meshlab_script_mid = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
input_mesh = path_and_filename + ".off"


mid_output_return = meshlab_generic(key,meshlab_script_mid,input_mesh,mid_ouptut)
#path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"] + "_non_manifold"),meshfix.v,meshfix.f)
print(f"Step 3b: meshlab making manifold done: {time.time() - start_time}")

#Cleanse the mesh
start_time = time.time()
output_mesh = path_and_filename + "_mls_no_float.off"
meshlab_script_output = str(pathlib.Path.cwd()) + "/" + "remove_floating_pieces.mls"

output_mesh = meshlab_generic(key,meshlab_script_output,mid_ouptut,output_mesh)
#path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"] + "_non_manifold"),meshfix.v,meshfix.f)
print(f"Step 3c: meshlab remove floating arifacts: {time.time() - start_time}")


print(output_mesh[:-4])
#send to be skeletonized
start_time = time.time()
return_value = cm.calcification(output_mesh[:-4])
if return_value > 0:
    print('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                    ' did not finish... exited with error code: ' + str(return_value))
print(f"Step 4: Generating Skeleton: {time.time() - start_time}")

In [ ]:
#new_id = 648518346349483124

In [ ]:
#neur_data = (ta3p100.VoxelizedMesh & "segment_id=648518346349483124").fetch1()